In [1]:
import geopandas as gpd
import json

In [2]:
import asyncio
import logging
import sys

import httpx
import requests
from requests.exceptions import RequestException
from tenacity import before_sleep_log, retry, retry_if_exception_type, stop_after_attempt, wait_random_exponential

In [3]:
from json import JSONDecodeError
import xmltodict

In [4]:
from requests.exceptions import HTTPError

class ResponseNotJson(HTTPError):
    '''Raised when the response is not a JSON'''

class ResponseNotXML(HTTPError):
    '''Raised when the response is not a XML'''

class PaginationError(HTTPError):
    '''Raised hen total retrieved features is not equal to total features returned by API'''

In [5]:
def raise_for_status(func):
    '''Raises HTTP error for response status codes 4xx or 5xx.
    If not, returns content of response'''

    def decorated(*args, **kwargs):

        response = func(*args, **kwargs)
        response.raise_for_status()
        content = response.content

        return content

    return decorated


def json_response(func):
    '''Parses json response. Raises xml string error if
    xml is returned'''

    def decorated(*args, **kwargs):

        response = func(*args, **kwargs)
        json_txt = response.decode('utf-8')
        try:
            json_data = json.loads(json_txt)
            return json_data
        except JSONDecodeError:
            response = xmltodict.parse(response)
            raise ResponseNotJson(f'Response is not a JSON: {response}')
    
    return decorated

def xml_response(func):
    '''Parses xml response as a dict'''

    def decorated(*args, **kwargs):
        
        try:
            response = func(*args, **kwargs)
            parsed = xmltodict.parse(response)
            return parsed
        except Exception as e:
            raise ResponseNotXML(f'XML parsing failed {e}')
    
    return decorated

In [6]:
logger = logging.getLogger(__name__)

class BaseClient:
    """WFS base client - used to make generic requests"""

    accepted_versions = {"2.0.0"}
    output_formats = {"json", "xml", "bytes"}

    def __init__(self, domain: str, version: str = "2.0.0", verbose: bool = True):
        self.domain = self.__clean_domain(domain)
        self.version = self.__assert_version(version)
        self.host = self.__gen_host()
        self.verbose = verbose

    def __clean_domain(self, domain: str) -> str:
        if domain.endswith(r"/"):
            domain = domain[:-1]

        return domain

    def __assert_version(self, version: str) -> str:
        if version not in self.accepted_versions:
            raise ValueError(f"Accepted versions: {self.accepted_versions}")
        return version

    def __gen_host(self) -> str:
        wfs_version = f"/?service=WFS&version={self.version}"
        return self.domain + wfs_version

    def __solve_get_params(self, *ignore, **query_params: dict) -> str:
        request_args = ["=".join([param_name, str(param_value)]) for param_name, param_value in query_params.items()]
        query_string = "&".join(request_args)

        return query_string

    def __solve_req_capability(self, capability: str) -> str:
        capability_param = f"request={capability}"

        return capability_param

    def __solve_request_url(self, capability: str, **query_params: dict) -> str:
        base_url = self.host
        capability_param = self.__solve_req_capability(capability)
        url = base_url + "&" + capability_param

        if query_params is None:
            return url
        else:
            req_params = self.__solve_get_params(**query_params)
            return url + "&" + req_params

    @retry(
        retry=retry_if_exception_type(RequestException),
        stop=stop_after_attempt(10),
        wait=wait_random_exponential(5, min=5, max=60),
        before_sleep=before_sleep_log(logger, logging.INFO, exc_info=True),
    )
    @raise_for_status
    def wfs_generic_request(self, capability: str, *ignore, **query_params: dict) -> bytes:
        url = self.__solve_request_url(capability, **query_params)

        # response is in bytes, so must be decoed accordingly
        with requests.get(url) as response:
            return response

    async def wfs_async_requests(self, capability: str, pages, **query_params: dict):
        params = query_params.copy()
        async with httpx.AsyncClient() as client:

            @retry(
                retry=retry_if_exception_type(httpx.HTTPError),
                stop=stop_after_attempt(10),
                wait=wait_random_exponential(30, min=30, max=300),
                before_sleep=before_sleep_log(logger, logging.INFO, exc_info=True),
            )
            async def fetch(url, semaphore):
                async with semaphore:
                    response = await client.get(url, timeout=300)
                    response.raise_for_status()
                    return response

            tasks = []
            semaphore = asyncio.Semaphore(4)

            for page in pages:
                params["startIndex"] = page
                url = self.__solve_request_url(capability, **params)
                task = asyncio.create_task(fetch(url, semaphore))
                tasks.append(task)

            responses = await asyncio.gather(*tasks, return_exceptions=True)

        for response in responses:
            if isinstance(response, Exception):
                # Handle exceptions
                print(f"Request failed: {response}")

        @json_response
        def parse_response(response):
            return response.content

        responses = [parse_response(response) for response in responses]

        first_response = responses[0]
        for response in responses[1:]:
            first_response["features"].extend(response["features"])

        return first_response

    @json_response
    def get_json(self, capability: str, **query_params: dict) -> dict:
        return self.wfs_generic_request(
            capability, outputFormat="application/json", exceptions="application/json", **query_params
        )

    @xml_response
    def get_xml(self, capability: str, **query_params: dict) -> dict:
        return self.wfs_generic_request(capability, **query_params)

    def __call__(self, capability: str, *ignore, output_format="json", pages=None, **query_params: dict) -> bytes:
        if output_format not in self.output_formats:
            raise ValueError(f"output_format must be in {self.output_formats}")

        if output_format == "json":
            return self.get_json(capability, **query_params)

        elif output_format == "xml":
            return self.get_xml(capability, **query_params)

        else:
            return self.wfs_generic_request(capability, **query_params)


In [7]:
from typing import Union


class CQLFilter:
    '''Builds a CQL filter for querying the WFS server'''
    
    def __init__(self, feature_name:str, schema:dict)->None:
        
        self.feature_name = feature_name
        self.schema = schema[feature_name]
        self.cql_filter = ''
        
    def add_to_filter(self, query_str:str, sep:str=';')->None:
        
        if len(self.cql_filter)<1:
            self.cql_filter=query_str
        else:
            query_str = sep + query_str
            self.cql_filter += query_str
    
    def __check_propertie_in_schema(self, prop_name:str)->None:
        
        if prop_name not in self.schema:
            raise ValueError(f'Feature name {prop_name} must be in {self.schema.keys()}')
        
    def __propertie_equals(self, propertie_name:str, equals_to:Union[str, int, float]):
        
        self.__check_propertie_in_schema(propertie_name)
        if type(equals_to) is str:
            equals_to = f"'{equals_to}'"

        query_str = f"({propertie_name}={equals_to})"
        
        return query_str
    
    def properties_equals(self, *ignore, **propertie_comparisons):
        
        #se tiver mais de um filtro, separa corretamente
        self.add_to_filter('', sep=';')
        for prop_name, prop_val in propertie_comparisons.items():
            query_str = self.__propertie_equals(prop_name, prop_val)
            self.add_to_filter(query_str, sep='AND')

    def point_within_pol(self, x:float, y:float, precision:int=5)->dict:
        '''Precision is in meters. Returns the polygon in self.feature_name 
        which intersects a buffer of {precision} meters centralized at the point'''

        query = f'DWITHIN(ge_poligono,POINT({x} {y}),{precision},meters)'
        self.add_to_filter(query)

    def point_within_linha(self, x:float, y:float, precision:int=5)->dict:
        '''Precision is in meters. Returns the polygon in self.feature_name 
        which intersects a buffer of {precision} meters centralized at the point'''

        query = f'DWITHIN(ge_linha,POINT({x} {y}),{precision},meters)'
        self.add_to_filter(query)

    def point_within_multipol(self, x:float, y:float, precision:int=5)->dict:
        '''Precision is in meters. Returns the polygon in self.feature_name 
        which intersects a buffer of {precision} meters centralized at the point'''

        query = f'DWITHIN(ge_multipoligono,POINT({x} {y}),{precision},meters)'
        self.add_to_filter(query)

    def polygon_within_pol(self, coordinates:str, precision:int=5)->dict:
        '''Precision is in meters. Returns the polygon in self.feature_name 
        which intersects a buffer of {precision} meters centralized at the point'''

        query = f'DWITHIN(ge_poligono,POLYGON({coordinates}),{precision},meters)'
        self.add_to_filter(query)

    def polygon_within_linha(self, coordinates:str, precision:int=5)->dict:
        '''Precision is in meters. Returns the polygon in self.feature_name 
        which intersects a buffer of {precision} meters centralized at the point'''

        query = f'DWITHIN(ge_linha,POLYGON({coordinates}),{precision},meters)'
        self.add_to_filter(query)

    def polygon_within_multipol(self, coordinates:str, precision:int=5)->dict:
        '''Precision is in meters. Returns the polygon in self.feature_name 
        which intersects a buffer of {precision} meters centralized at the point'''

        query = f'DWITHIN(ge_multipoligono,POLYGON({coordinates}),{precision},meters)'
        self.add_to_filter(query)


    def __call__(self):
        
        return self.cql_filter

In [8]:
import warnings

In [9]:
class Paginator:
    def __init__(self, get_function: BaseClient, schemas: dict = None) -> None:
        self.schemas = schemas or {}
        self.get = get_function

    def extract_total_features(self, resp: dict) -> int:
        return resp["totalFeatures"]

    def extract_returned_quantity(self, resp: dict) -> int:
        return len(resp["features"])

    def needs_pagination(self, total_features: int, returned_quantity: int) -> bool:
        return total_features > returned_quantity

    def get_steps(self, total_features: int, returned_quantity: int) -> list:
        # need to start at zero because first query wans't ordered
        return [step for step in range(0, total_features, returned_quantity)]

    def warn_steps(self, feature_name: str, steps: list) -> None:
        total_steps = len(steps)

        warnings.warn(f"Paginação iniciada. Serão realizadas {total_steps} requisições para a {feature_name}")

    def get_index_col(self, feature_name: str, index_col: str = None) -> str:
        if index_col:
            warnings.warn(f"Certifique-se que a index_col de fato é uma coluna existente na feature")
            return index_col

        if not self.schemas:
            raise ValueError(f"Must set schemas if willing to paginate and not specify index col")

        feature_schemas = self.schemas.get(feature_name, None)
        if feature_schemas is None:
            raise ValueError(f"Schema for feature {feature_name} not found. Must specify index col")

        index_col = feature_schemas.get("id_col")
        if index_col is None:
            raise ValueError(f"Feature {feature_name} has no defaul index col. Must specify index col")

        return index_col

    def paginate(self, feature_name: str, resp: dict, index_col: str = None, *_ignored, **query_params) -> dict:
        total_features = self.extract_total_features(resp)
        returned_quantity = self.extract_returned_quantity(resp)

        if not self.needs_pagination(total_features, returned_quantity):
            return resp

        print("Paginação iniciada")
        index_col = self.get_index_col(feature_name, index_col)

        steps = self.get_steps(total_features, returned_quantity)
        self.warn_steps(feature_name, steps)

        # recriando as features
        resp["features"] = []

        for _ in steps:
            resp_step = self.get("GetFeature", typeName=feature_name, sortBy=index_col, **query_params)
            features_step = resp_step["features"]
            resp["features"].extend(features_step)
            max_index = features_step[-1]["properties"][index_col]
            query_params["cql_filter"] = f"{index_col}>{max_index}"

        total_returned_features = len(resp["features"])
        if not total_returned_features == total_features:
            raise PaginationError(
                f"""Difference in total features and paginated features: 
                          total - {total_features} vs returned - {total_returned_features}
                          """
            )

        return resp

    def __call__(self, feature_name: str, resp: dict, index_col: str = None, *_ignored, **query_params) -> dict:
        return self.paginate(feature_name, resp, index_col, **query_params)


In [10]:
import warnings
from collections import OrderedDict

class FeatureMdataParser:
    '''Parses feature metadata'''

    def parse_property(self, prop:dict)->dict:

        name = prop['name']
        parsed = dict(
            nullable = prop['nillable'],
            dtype = prop['localType']
            )

        return {name : parsed}
    
    def get_cd_identificador(self, prop:dict, cd_col_name:str='cd_identifica')->bool:

        name = prop['name']
        if name.lower().startswith(cd_col_name):
            return True
        return False
    
    def set_cd_identificador(self, properties:dict, prop:dict)->None:

        if self.get_cd_identificador(prop) and \
            not properties.get('id_col'):

            properties['id_col'] = prop['name']

    def parse_feature_schema(self, feat:dict)->dict:

        name = feat['typeName']
        properties = OrderedDict()
        for prop in feat['properties']:
            parsed_prop = self.parse_property(prop)
            properties.update(parsed_prop)

            self.set_cd_identificador(properties, prop)

        parsed ={
            name : properties
        }

        return parsed
    
    def raise_for_no_id(self, parsed_feature:dict)->None:

        for feature_name, mdata in parsed_feature.items():
            if 'id_col' not in mdata:
                warnings.warn(f"Could not identify id col for feature {feature_name}")

    def parse_all_features(self, get_feature_resp:dict)->dict:

        features = get_feature_resp['featureTypes']

        parsed = OrderedDict()
        for feat in features:
            parsed_feat = self.parse_feature_schema(feat)
            parsed.update(parsed_feat)
            self.raise_for_no_id(parsed_feat)
        return parsed

    def __call__(self, get_feature_resp:dict)->dict:

        return self.parse_all_features(get_feature_resp)

In [11]:
class GeoSampaWfs(BaseClient):

    def __init__(self, domain:str, set_schemas:bool=True, verbose:bool=True, auto_paginate:bool=True):

        self.get = BaseClient(domain=domain, verbose=verbose)
        self.parse_features_schemas = FeatureMdataParser()

        self.schemas = self.get_feature_schemas() if set_schemas else None
        self.paginate = Paginator(self.get, self.schemas)

        self.auto_paginate = auto_paginate
    
    def __list_feature_types_raw(self):

        return self.get('DescribeFeatureType')

    def get_feature_schemas(self):

        resp  = self.__list_feature_types_raw()
        return self.parse_features_schemas(resp)

    def __solve_properties(self, query_params, properties:list=None):

         if properties:
            names = ','.join(properties)
            query_params['propertyName']=names

    def __check_feature_exists(self, feature_name:str)->None:

        if self.schemas and feature_name not in self.schemas:
            raise ValueError(f"Feature name {feature_name} doesn't exits")

    def get_feature(self, feature_name:str, properties:list=None, filter:CQLFilter=None, paginate:bool=None,
                    index_col:str=None, **query_params):

        self.__check_feature_exists(feature_name)        
        self.__solve_properties(query_params, properties)
       
        if filter is not None:
            query_params['cql_filter'] = filter()

        resp = self.get('GetFeature', typeName=feature_name, **query_params)

        if paginate or (paginate is None and self.auto_paginate):
            return self.paginate(feature_name, resp, index_col, **query_params)

In [12]:
import os
import shutil
import warnings

from dotenv import load_dotenv

In [13]:
#ROOT_DIR = os.path.dirname(os.path.abspath(__file__))
ROOT_DIR = os.path.dirname(os.path.abspath(''))
print (ROOT_DIR)

F:\General Projects\Git\GitHub\sepep-pmsp\saade_governancadados


In [14]:
class MissingEnvironmentVariable(RuntimeError):
    pass

def get_dotenv_path(root: str = ROOT_DIR, example: bool = False) -> str:
    """Returns .env file path"""

    path = os.path.join(root, ".env")

    if example:
        path += ".example"

    return path


def dotenv_exits(dotenv_path: str = None, root: str = ROOT_DIR):
    """Check if .env file path exists"""

    dotenv_path = dotenv_path or get_dotenv_path(root)

    return os.path.exists(dotenv_path)


def solve_dot_env(root: str = ROOT_DIR) -> str:
    """Copies .env.example as .env if .env doesnt exists"""

    dotenv_path = get_dotenv_path(root)

    if not dotenv_exits(dotenv_path):
        warnings.warn(".env file not found: copying .env.example")
        env_example = get_dotenv_path(root, example=True)
        shutil.copy(env_example, dotenv_path)

    return dotenv_path


def load_env(root: str = ROOT_DIR) -> None:
    """Loads .env. If .env doesn't exists, will save .env.example
    as .env and will load it's variables"""

    env_path = solve_dot_env(root)
    load_dotenv(dotenv_path=env_path)


def load_var(varname: str, root: str = ROOT_DIR) -> str:
    load_env(root)

    try:
        return os.environ[varname]
    except KeyError:
        raise MissingEnvironmentVariable(f"Environment var {varname} not defined")


In [15]:
GEOSAMPA_WFS_DOMAIN = load_var('GEOSAMPA_WFS_DOMAIN')

def get_client(domain=GEOSAMPA_WFS_DOMAIN):
    print (GEOSAMPA_WFS_DOMAIN)

    return GeoSampaWfs(domain)

In [16]:
geosampa = get_client()

http://wfs.geosampa.prefeitura.sp.gov.br/geoserver/ows


C:\Users\leona\AppData\Local\Temp\ipykernel_42424\2719722458.py:51: UserWarning: Could not identify id col for feature geohabisp_vw_wfs_cortico_geosampa
  warnings.warn(f"Could not identify id col for feature {feature_name}")
C:\Users\leona\AppData\Local\Temp\ipykernel_42424\2719722458.py:51: UserWarning: Could not identify id col for feature geohabisp_vw_wfs_favela_geosampa
  warnings.warn(f"Could not identify id col for feature {feature_name}")
C:\Users\leona\AppData\Local\Temp\ipykernel_42424\2719722458.py:51: UserWarning: Could not identify id col for feature geohabisp_vw_wfs_loteamento_geosampa
  warnings.warn(f"Could not identify id col for feature {feature_name}")
C:\Users\leona\AppData\Local\Temp\ipykernel_42424\2719722458.py:51: UserWarning: Could not identify id col for feature geohabisp_vw_wfs_nucleo_geosampa
  warnings.warn(f"Could not identify id col for feature {feature_name}")


In [17]:
schemas = geosampa.get_feature_schemas()

C:\Users\leona\AppData\Local\Temp\ipykernel_42424\2719722458.py:51: UserWarning: Could not identify id col for feature geohabisp_vw_wfs_cortico_geosampa
  warnings.warn(f"Could not identify id col for feature {feature_name}")
C:\Users\leona\AppData\Local\Temp\ipykernel_42424\2719722458.py:51: UserWarning: Could not identify id col for feature geohabisp_vw_wfs_favela_geosampa
  warnings.warn(f"Could not identify id col for feature {feature_name}")
C:\Users\leona\AppData\Local\Temp\ipykernel_42424\2719722458.py:51: UserWarning: Could not identify id col for feature geohabisp_vw_wfs_loteamento_geosampa
  warnings.warn(f"Could not identify id col for feature {feature_name}")
C:\Users\leona\AppData\Local\Temp\ipykernel_42424\2719722458.py:51: UserWarning: Could not identify id col for feature geohabisp_vw_wfs_nucleo_geosampa
  warnings.warn(f"Could not identify id col for feature {feature_name}")


In [18]:
dict_exemplos = []

cat = []
cat.append({ 'Zoneamento Geral': { 'camada' : 'pde2014_v_zu_zr_01a_map', 'prefixo_campo': 'geom_macrozoneamento_zu_zr_01a' } })
cat.append({ 'Macrozonas': { 'camada' : 'pde2014_v_mcrz_01_map' , 'prefixo_campo': 'geom_macrozoneamento_mcrz_01' } } )
cat.append({ 'Macroáreas': { 'camada' : 'pde2014_v_mcrar_02_map' , 'prefixo_campo': 'geom_macrozoneamento_mcrar_02' } })
cat.append({ 'Setores': { 'camada' : 'pde2014_v_maem_02a_map' , 'prefixo_campo': 'geom_macrozoneamento_maem_02a' } })
cat.append({ 'Eixos Existentes': { 'camada' : 'pde2014_v_eetr_03_map' , 'prefixo_campo': 'geom_macrozoneamento_eetr_03' } })
cat.append({ 'Eixos Previstos': { 'camada' : 'pde2014_v_eixo_prv_03a_map' , 'prefixo_campo': 'geom_macrozoneamento_eixo_prv_03a' } })
cat.append({ 'Eixos Previstos - Ativados por Decreto': { 'camada' : 'eixo_ativado_decreto' , 'prefixo_campo': 'geom_macrozoneamento_eixo_ativado_decreto' } })

dict_exemplos.append({
        'identificador': 'geom_macrozoneamento'
        , 'categorias': tuple(cat)
        , 'sheet_name' : 'geom_macrozoneamento'
        ,
    })


cat = []
cat.append({ 'ZEIS': { 'camada' : 'pde2014_v_zeis_04_map', 'prefixo_campo': 'geom_zeis-pde_4' } })
cat.append({ 'ZEIS': { 'camada' : 'pde2014_v_zeis_04a_map' , 'prefixo_campo': 'geom_zeis-pde_4a' } } )
dict_exemplos.append(
    {
        'identificador': 'geom_zeis-pde'
        , 'categorias': tuple(cat)
        , 'sheet_name' : 'geom_zeis-pde'
        ,
    })

cat = []
cat.append({ 'Zoneamento': { 'camada' : 'zoneamento_2016_map1', 'prefixo_campo': 'geom_zoneamento_perimetro' } })
cat.append({ 'Zoneamento - Qualificação Ambiental': { 'camada' : None , 'prefixo_campo': 'geom_zoneamento_quali_amb' } } )
cat.append({ 'Zoneamento - Incentivos': { 'camada' : None , 'prefixo_campo': 'geom_zoneamento_incentivo_edif_garag' } } )
dict_exemplos.append(
    {
        'identificador': 'geom_zoneamento'
        , 'categorias': tuple(cat)
        , 'sheet_name' : 'geom_zoneamento'
        ,
    })

cat = []
cat.append({ 'Operação Urbana': { 'camada' : 'operacao_urbana', 'prefixo_campo': 'geom_operacao_urbana' } })
dict_exemplos.append(
    {
        'identificador': 'geom_operacao_urbana'
        , 'categorias': tuple(cat)
        , 'sheet_name' : 'geom_operacao_urbana'
        ,
    })

cat = []
cat.append({ 'Área de Intervenção Urbana': { 'camada' : 'perimetro_aiu', 'prefixo_campo': 'geom_aiu_perimetro' } })
cat.append({ 'Área de Intervenção Urbana - Adesão': { 'camada' : 'aiu_vl_perimetro_adesao', 'prefixo_campo': 'geom_aiu_vl_perimetro_adesao' } })
dict_exemplos.append(
    {
        'identificador': 'geom_aiu'
        , 'categorias': tuple(cat)
        , 'sheet_name' : 'geom_aiu'
        ,
    })

cat = []
cat.append({ 'Leis Esparsas - Perimetro Geral': { 'camada' : 'requalifica_centro_perimetro_geral', 'prefixo_campo': 'geom_leis_esparsas_requalifica_centro_perimetro_geral' } })
cat.append({ 'Leis Esparsas - Perimetro Especial': { 'camada' : 'requalifica_centro_perimetro_especial', 'prefixo_campo': 'geom_leis_esparsas_requalifica_centro_perimetro_especial' } })
cat.append({ 'Leis Esparsas - Mirante Santana': { 'camada' : 'restricao_mirante_santana', 'prefixo_campo': 'geom_leis_esparsas_restricao_mirante_santana' } })
dict_exemplos.append(
    {
        'identificador': 'geom_leis_esparsas'
        , 'categorias': tuple(cat)
        , 'sheet_name' : 'geom_leis_esparsas'
        ,
    })

cat = []
cat.append({ 'Sub Prefeituras': { 'camada' : 'subprefeitura', 'prefixo_campo': 'geom_subprefeitura' } })
dict_exemplos.append(
    {
        'identificador': 'geom_subprefeitura'
        , 'categorias': tuple(cat)
        , 'sheet_name' : 'geom_subprefeitura'
        ,
    })

cat = []
cat.append({ 'Distritos': { 'camada' : 'distrito_municipal', 'prefixo_campo': 'geom_distrito' } })
dict_exemplos.append(
    {
        'identificador': 'geom_distrito'
        , 'categorias': tuple(cat)
        , 'sheet_name' : 'geom_distrito'
        ,
    })

cat = []
cat.append({ 'Melhoramento Viário': { 'camada' : 'geoconvias_lei_melhoramento_vigente', 'prefixo_campo': 'geom_distrito' } })
dict_exemplos.append(
    {
        'identificador': 'geom_melhoramento_viario'
        , 'categorias': tuple(cat)
        , 'sheet_name' : 'geom_melhoramento_viario'        
        ,
    })

cat = []
cat.append({ 'Decretos de Declaração de Interesse Social ou de Utilidade Pública': { 'camada' : 'decreto_utilidade_publica_interesse_social', 'prefixo_campo': 'geom_dis_dup' } })
dict_exemplos.append(
    {
        'identificador': 'geom_dis_dup'
        , 'categorias': tuple(cat)
        , 'sheet_name' : 'geom_dis_dup'
        ,
    })

cat = []
cat.append({ 'Faixas de Domínio - Dutovias': { 'camada' : 'transpetro_duto', 'prefixo_campo': 'geom_faixas_dominio_duto_oleo_gas' } })
cat.append({ 'Faixas de Domínio - Ferrovias - Linha': { 'camada' : 'linha_trem', 'prefixo_campo': 'geom_faixas_dominio_trem_linha' } })
cat.append({ 'Faixas de Domínio - Ferrovias': { 'camada' : 'ferrovia_mdc', 'prefixo_campo': 'geom_faixas_dominio_trem_ferrovia' } })
cat.append({ 'Faixas de Domínio - Ferrovias - Metrô': { 'camada' : 'linha_metro', 'prefixo_campo': 'geom_faixas_dominio_linha_metro' } })
cat.append({ 'Faixas de Domínio - Estradas e Rodovias': { 'camada' : None, 'prefixo_campo': 'geom_faixas_dominio_estrada_rodovia' } })
cat.append({ 'Faixas de Domínio - Linhas de Transmissão': { 'camada' : None, 'prefixo_campo': 'geom_faixas_dominio_linha_transmissao' } })
dict_exemplos.append(
    {
        'identificador': 'geom_faixas_dominio'
        , 'categorias': tuple(cat)
        , 'sheet_name' : 'geom_faixas_dominio'
        ,
    })

cat = []
cat.append({ 'Terra Indígena': { 'camada' : 'GEOSAMPA_terra_indigena', 'prefixo_campo': 'geom_terra_indigena' } })
dict_exemplos.append(
    {
        'identificador': 'geom_terra_indigena'
        , 'categorias': tuple(cat)
        , 'sheet_name' : 'geom_terra_indigena'
        ,
    })

cat = []
cat.append({ 'Áreas Públicas': { 'camada' : 'calcada', 'prefixo_campo': 'geom_areas_publicas' } })
dict_exemplos.append(
    {
        'identificador': 'geom_areas_publicas'
        , 'categorias': tuple(cat)
        , 'sheet_name' : 'geom_areas_publicas'
        ,
    })

cat = []
cat.append({ 'Faixa Não Edificável - Dutovias': { 'camada' : 'transpetro_duto', 'prefixo_campo': 'geom_faixas_nao_edificaveis_duto_oleo_gas' } })
cat.append({ 'Faixa Não Edificável - Ferrovias - Linha': { 'camada' : 'linha_trem', 'prefixo_campo': 'geom_faixas_nao_edificaveis_trem_linha' } })
cat.append({ 'Faixa Não Edificável - Ferrovias': { 'camada' : 'ferrovia_mdc', 'prefixo_campo': 'geom_faixas_nao_edificaveis_trem_ferrovia' } })
cat.append({ 'Faixa Não Edificável - Ferrovias - Metrô': { 'camada' : 'linha_metro', 'prefixo_campo': 'geom_faixas_nao_edificaveis_linha_metro' } })
cat.append({ 'Faixa Não Edificável - Estradas e Rodovias': { 'camada' : None, 'prefixo_campo': 'geom_faixas_nao_edificaveis_estrada_rodovia' } })
cat.append({ 'Faixa Não Edificável - Equipamentos Públicos': { 'camada' : None, 'prefixo_campo': 'geom_faixas_nao_edificaveis_equip_publico' } })
cat.append({ 'Faixa Não Edificável - Equipamentos Públicos Arruamento': { 'camada' : 'decreto_utilidade_publica_interesse_social', 'prefixo_campo': 'geom_faixas_nao_edificaveis_equip_publico_arr' } })
cat.append({ 'Faixa Não Edificável - Equipamentos Públicos Área Urbana Regularizada': { 'camada' : 'decreto_utilidade_publica_interesse_social', 'prefixo_campo': 'geom_faixas_nao_edificaveis_equip_publico_au' } })
cat.append({ 'Faixa Não Edificável - Melhoramentos Públicos': { 'camada' : None, 'prefixo_campo': 'geom_faixas_nao_edificaveis_melhora_publico' } })
cat.append({ 'Faixa Não Edificável - Águas Correntes ou Dormentes - Drenagem - Estado Natural': { 'camada' : 'drenagem', 'prefixo_campo': 'geom_faixas_nao_edificaveis_agua_corr_dormente_drena_natural' } })
cat.append({ 'Faixa Não Edificável - Águas Correntes ou Dormentes - Drenagem - Lago/Reservatório': { 'camada' : 'drenagem', 'prefixo_campo': 'geom_faixas_nao_edificaveis_agua_corr_dormente_drena_lago_res' } })
cat.append({ 'Faixa Não Edificável - Águas Correntes ou Dormentes - Massa D''água': { 'camada' : 'massa_d_agua', 'prefixo_campo': 'geom_faixas_nao_edificaveis_agua_corr_dormente_massa_dagua' } })
cat.append({ 'Faixa Não Edificável - Águas Correntes ou Dormentes - Represa': { 'camada' : 'represa_nivel_maximo', 'prefixo_campo': 'geom_faixas_nao_edificaveis_agua_corr_dormente_represa' } })
cat.append({ 'Faixa Não Edificável - Galerias e Canalizações - Infraestrutura': { 'camada' : 'geoconvias_faixa_nao_edificavel', 'prefixo_campo': 'geom_faixas_nao_edificaveis_galeria_canal_infraest' } })
cat.append({ 'Faixa Não Edificável - Galerias e Canalizações - Drenagem - Céu aberto': { 'camada' : 'drenagem', 'prefixo_campo': 'geom_faixas_nao_edificaveis_galeria_canal_drena_ceu_aberto' } })
cat.append({ 'Faixa Não Edificável - Galerias e Canalizações - Drenagem - Subterrânea': { 'camada' : 'drenagem', 'prefixo_campo': 'geom_faixas_nao_edificaveis_galeria_canal_drena_subterra' } })
cat.append({ 'Faixa Não Edificável - Galerias e Canalizações - Massa D''água': { 'camada' : 'massa_d_agua', 'prefixo_campo': 'geom_faixas_nao_edificaveis_galeria_canal_massa_dagua' } })
cat.append({ 'Faixa Não Edificável - Linhas de Transmissão': { 'camada' : None, 'prefixo_campo': 'geom_faixas_nao_edificaveis_linha_transmissao' } })

dict_exemplos.append(
    {
        'identificador': 'geom_faixas_nao_edificaveis'
        , 'categorias': tuple(cat)
        , 'sheet_name' : 'geom_faixas_nao_edificaveis'
        ,
    })

cat = []
cat.append({ 'Cartório de Registro de Imóveis': { 'camada' : 'cartorio_registro_imovel', 'prefixo_campo': 'geom_cartorio_registro_imoveis' } })
dict_exemplos.append(
    {
        'identificador': 'geom_cartorio_registro_imoveis'
        , 'categorias': tuple(cat)
        , 'sheet_name' : 'geom_cartorio_registro_imoveis'
        ,
    })

cat = []
cat.append({ 'Espaço Aéreo - Heliponto': { 'camada' : 'GEOSAMPA_heliponto', 'prefixo_campo': 'geom_espaco_aereo_heliponto' } })
cat.append({ 'Espaço Aéreo - Rota de Aviões e Helicópteros': { 'camada' : None, 'prefixo_campo': 'geom_espaco_aereo_rota_avioes_helicopteros' } })
cat.append({ 'Espaço Aéreo - Plano': { 'camada' : None, 'prefixo_campo': 'geom_espaco_aereo_plano' } })
cat.append({ 'Espaço Aéreo - Auxílio à Navegação - Ponto': { 'camada' : None, 'prefixo_campo': 'geom_espaco_aereo_auxilio_navegacao_ponto' } })
cat.append({ 'Espaço Aéreo - Auxílio à Navegação - Linha': { 'camada' : None, 'prefixo_campo': 'geom_espaco_aereo_auxilio_navegacao_linha' } })
cat.append({ 'Espaço Aéreo - Auxílio à Navegação': { 'camada' : None, 'prefixo_campo': 'geom_espaco_aereo_auxilio_navegacao' } })
dict_exemplos.append(
    {
        'identificador': 'geom_espaco_aereo'
        , 'categorias': tuple(cat)
        , 'sheet_name' : 'geom_espaco_aereo'
        ,
    })

cat = []
cat.append({ 'Mini Anel Viário': { 'camada' : 'restricao_circulacao_veiculo_mian', 'prefixo_campo': 'geo_minianel_viario' } })
dict_exemplos.append(
    {
        'identificador': 'geo_minianel_viario'
        , 'categorias': tuple(cat)
        , 'sheet_name' : 'geo_minianel_viario'
        ,
    })

cat = []
cat.append({ 'Logradouro': { 'camada' : None, 'prefixo_campo': 'geom_logradouro_identificacao_logradouro' } })
cat.append({ 'Logradouro SF': { 'camada' : None, 'prefixo_campo': 'geom_logradouro_identificacao_logradouro_sf' } })
cat.append({ 'Classificação urbanística - Quadro 9.N1': { 'camada' : 'classificacao_viaria_quadro_9', 'prefixo_campo': 'geom_logradouro_class_viaria_q9n1' } })
cat.append({ 'Classificação urbanística - Quadro 9.N2': { 'camada' : 'classificacao_viaria_quadro_9', 'prefixo_campo': 'geom_logradouro_class_viaria_q9n2' } })
cat.append({ 'Classificação urbanística - Quadro 9.N3': { 'camada' : 'classificacao_viaria_quadro_9', 'prefixo_campo': 'geom_logradouro_class_viaria_q9n3' } })
cat.append({ 'Classificação urbanística - Coletora CET': { 'camada' : 'classificacao_viaria_cet', 'prefixo_campo': 'geom_logradouro_class_viaria_cet_coletora' } })
cat.append({ 'Classificação urbanística - Local CET': { 'camada' : 'classificacao_viaria_cet', 'prefixo_campo': 'geom_logradouro_class_viaria_cet_local' } })
cat.append({ 'Classificação urbanística - Vias Pedestre CET': { 'camada' : 'classificacao_viaria_cet', 'prefixo_campo': 'geom_logradouro_class_viaria_cet_via_pedestre' } })
cat.append({ 'Estradas e Rodovias': { 'camada' : None, 'prefixo_campo': 'geom_logradouro_estrada_rodovia' } })
cat.append({ 'Vias relacionadas a zonas': { 'camada' : None, 'prefixo_campo': 'geom_logradouro_via_relac_zonas' } })
dict_exemplos.append(
    {
        'identificador': 'geom_logradouro'
        , 'categorias': tuple(cat)
        , 'sheet_name' : 'geom_logradouro'
        ,
    })

cat = []
cat.append({ 'Tipo de Logradouro': { 'camada' : None, 'prefixo_campo': 'geom_tipo_logradouro' } })
dict_exemplos.append(
    {
        'identificador': 'geom_tipo_logradouro' #CADLOG
        , 'categorias': tuple(cat)
        , 'sheet_name' : 'geom_tipo_logradouro'
        ,
    })

cat = []
cat.append({ 'Face da Quadra': { 'camada' : None, 'prefixo_campo': 'geom_face_quadra' } })
dict_exemplos.append(
    {
        'identificador': 'geom_face_quadra'
        , 'categorias': tuple(cat)
        , 'sheet_name' : 'geom_face_quadra'
        ,
    })

cat = []
cat.append({ 'Rua Sem Saída': { 'camada' : None, 'prefixo_campo': 'geom_rua_sem_saida' } })
dict_exemplos.append(
    {
        'identificador': 'geom_rua_sem_saida'
        , 'categorias': tuple(cat)
        , 'sheet_name' : 'geom_rua_sem_saida'
        ,
    })

cat = []
cat.append({ 'Vila': { 'camada' : None, 'prefixo_campo': 'geom_vila' } })
dict_exemplos.append(
    {
        'identificador': 'geom_vila'
        , 'categorias': tuple(cat)
        , 'sheet_name' : 'geom_vila'
        ,
    })

cat = []
cat.append({ 'Entorno de Vila': { 'camada' : None, 'prefixo_campo': 'geom_entorno_vila' } })
dict_exemplos.append(
    {
        'identificador': 'geom_entorno_vila'
        , 'categorias': tuple(cat)
        , 'sheet_name' : 'geom_entorno_vila'
        ,
    })

cat = []
cat.append({ 'Curvas de Nível - Curva Mestra': { 'camada' : 'curva_mestra', 'prefixo_campo': 'geom_curvas_nivel_mestra' } })
cat.append({ 'Curvas de Nível - Curva Intermediária': { 'camada' : 'curva_intermediaria', 'prefixo_campo': 'geom_curvas_nivel_intermediaria' } })
dict_exemplos.append(
    {
        'identificador': 'geom_curvas_nivel'
        , 'categorias': tuple(cat)
        , 'sheet_name' : 'geom_curvas_nivel'
        ,
    })


cat = []
cat.append({ 'CEDI': { 'camada' : None, 'prefixo_campo': 'geom_cedi' } })
dict_exemplos.append(
    {
        'identificador': 'geom_cedi' #CEDI
        , 'categorias': tuple(cat)
        , 'sheet_name' : 'geom_cedi'
        ,
    })

"""
cat = []
cat.append({ 'Lote Fiscal': { 'camada' : 'lote_cidadao', 'prefixo_campo': 'geom_lote_fiscal' } })
dict_exemplos.append(
    {
        'identificador': 'geom_lote_fiscal'
        , 'categorias': tuple(cat)
        , 'sheet_name' : 'geom_lote_fiscal'
        ,
    })
"""


cat = []
cat.append({ 'TPCL': { 'camada' : None, 'prefixo_campo': 'geom_tpcl' } })
dict_exemplos.append(
    {
        'identificador': 'geom_tpcl' #TPCL
        , 'categorias': tuple(cat)
        , 'sheet_name' : 'geom_tpcl'
        ,
    })

cat = []
cat.append({ 'Billings - APPB': { 'camada' : 'manancial_billings', 'prefixo_campo': 'geom_app_billings_appb' } })
dict_exemplos.append(
    {
        'identificador': 'geom_app'
        , 'categorias': tuple(cat)
        , 'sheet_name': 'geom_app'
        ,
    })

cat = []
cat.append({ 'Unidade de Conservação - APA': { 'camada' : None, 'prefixo_campo': 'geom_uc_apa' } })
cat.append({ 'Unidade de Conservação - PE': { 'camada' : None, 'prefixo_campo': 'geom_uc_pe' } })
cat.append({ 'Unidade de Conservação - PNM': { 'camada' : None, 'prefixo_campo': 'geom_uc_pnm' } })
cat.append({ 'Unidade de Conservação - RPPN': { 'camada' : None, 'prefixo_campo': 'geom_uc_rppn' } })
dict_exemplos.append(
    {
        'identificador': 'geom_uc'
        , 'categorias': tuple(cat)
        , 'sheet_name': 'geom_uc'
        ,
    })

cat = []
cat.append({ 'Área de Proteção aos Mananciais': { 'camada' : None, 'prefixo_campo': 'geom_apm' } })
dict_exemplos.append(
    {
        'identificador': 'geom_apm'
        , 'categorias': tuple(cat)
        , 'sheet_name': 'geom_apm'
        ,
    })

cat = []
cat.append({ 'Área de Proteção de Recuperação de Mananciais - Billings': { 'camada' : 'manancial_billings', 'prefixo_campo': 'geom_aprm_billings' } })
cat.append({ 'Área de Proteção de Recuperação de Mananciais - Guarapiranga': { 'camada' : 'manancial_guarapiranga', 'prefixo_campo': 'geom_aprm_guarapiranga' } })
cat.append({ 'Área de Proteção de Recuperação de Mananciais - Ato Juquery': { 'camada' : 'manancial_juquery', 'prefixo_campo': 'geom_aprm_juquery' } })
dict_exemplos.append(
    {
        'identificador': 'geom_aprm'
        , 'categorias': tuple(cat)
        , 'sheet_name': 'geom_aprm'
        , 
    })

cat = []
cat.append({ 'Reserva Legal': { 'camada' : None, 'prefixo_campo': 'geom_reserva_legal' } })
dict_exemplos.append(
    {
        'identificador': 'geom_reserva_legal'
        , 'categorias': tuple(cat)
        , 'sheet_name': 'geom_reserva_legal'
        , 
    })

cat = []
cat.append({ 'Vegetação Árborea - Maçicos e Bosques': { 'camada' : 'cobertura_vegetal', 'prefixo_campo': 'geom_vegetacao_arborea_macicos_bosques' } })
cat.append({ 'Vegetação Árborea - Arbustiva e Arborescente': { 'camada' : 'cobertura_vegetal', 'prefixo_campo': 'geom_vegetacao_arborea_arbustiva_arborescente' } })
cat.append({ 'Vegetação Árborea - Viária/Árvores': { 'camada' : 'arvore', 'prefixo_campo': 'geom_vegetacao_arborea_viaria_arvores' } })
dict_exemplos.append(
    {
        'identificador': 'geom_vegetacao_arborea'
        , 'categorias': tuple(cat)
        , 'sheet_name': 'geom_vegetacao_arborea'
        , 
    })

cat = []
cat.append({ 'Bioma Mata Atlântica': { 'camada' : 'remanescente_pmma', 'prefixo_campo': 'geom_biomas_protegidos_mata_atlantica' } })
cat.append({ 'Bioma Cerrado': { 'camada' : None, 'prefixo_campo': 'geom_biomas_protegidos_cerrado' } })
dict_exemplos.append(
    {
        'identificador': 'geom_biomas_protegidos'
        , 'categorias': tuple(cat)
        , 'sheet_name': 'geom_biomas_protegidos'
        , 
    })

cat = []
cat.append({ 'Termo de Compromisso Ambiental': { 'camada' : 'termo_compromisso_ambiental_svma', 'prefixo_campo': 'geom_tca' } })
dict_exemplos.append(
    {
        'identificador': 'geom_tca'
        , 'categorias': tuple(cat)
        , 'sheet_name': 'geom_tca'
        , 
    })

cat = []
cat.append({ 'TAC Ambiental': { 'camada' : None, 'prefixo_campo': 'geom_tac_ambiental' } })
dict_exemplos.append(
    {
        'identificador': 'geom_tac_ambiental'
        , 'categorias': tuple(cat)
        , 'sheet_name': 'geom_tac_ambiental'
        , 
    })

cat = []
cat.append({ 'Vegetação em APP suprimida irregularmente': { 'camada' : None, 'prefixo_campo': 'geom_supressao_irregular_vegetacao_arborea_app_irr' } })
cat.append({ 'Vegetação em APP suprimida irregularmente recomposta': { 'camada' : None, 'prefixo_campo': 'geom_supressao_irregular_vegetacao_arborea_app_irr_rec' } })
cat.append({ 'Vegetação significativa suprimida irregularmente': { 'camada' : None, 'prefixo_campo': 'geom_supressao_irregular_vegetacao_arborea_irr' } })
cat.append({ 'Vegetação significativa suprimida irregularmente recomposta': { 'camada' : None, 'prefixo_campo': 'geom_supressao_irregular_vegetacao_arborea_irr_rec' } })
dict_exemplos.append(
    {
        'identificador': 'geom_supressao_irregular_vegetacao_arborea'
        , 'categorias': tuple(cat)
        , 'sheet_name': 'geom_supressao_irregular_vegetacao_arborea'
        , 
    })

cat = []
cat.append({ 'Área Contaminada': { 'camada' : 'area_contaminada_reabilitada_svma', 'prefixo_campo': 'geom_area_contaminada' } })
cat.append({ 'Área Contaminada e Reabilitada': { 'camada' : 'area_contaminada_reabilitada_svma', 'prefixo_campo': 'geom_area_contaminada_reabilitada' } })
dict_exemplos.append(
    {
        'identificador': 'geom_area_contaminada'
        , 'categorias': tuple(cat)
        , 'sheet_name': 'geom_area_contaminada'
        , 
    })

cat = []
cat.append({ 'Servidão Ambiental': { 'camada' : None, 'prefixo_campo': 'geom_servidao_ambiental' } })
dict_exemplos.append(
    {
        'identificador': 'geom_servidao_ambiental'
        , 'categorias': tuple(cat)
        , 'sheet_name': 'geom_servidao_ambiental'
        , 
    })

cat = []
cat.append({ 'Serviços Ambientais': { 'camada' : None, 'prefixo_campo': 'geom_servicos_ambientais' } })
dict_exemplos.append(
    {
        'identificador': 'geom_servicos_ambientais'
        , 'categorias': tuple(cat)
        , 'sheet_name': 'geom_servicos_ambientais'
        , 
    })

cat = []
cat.append({ 'Espaço Áereo - Ruído': { 'camada' : None, 'prefixo_campo': 'geom_espaço_aereo_ruido' } })
dict_exemplos.append(
    {
        'identificador': 'geom_espaço_aereo_ruido'
        , 'categorias': tuple(cat)
        , 'sheet_name': 'geom_espaço_aereo_ruido'
        , 
    })

cat = []
cat.append({ 'Risco Geológico': { 'camada' : '', 'prefixo_campo': 'geom_espaço_aereo_ruido' } })
cat.append({ 'Risco Hidrológico': { 'camada' : '', 'prefixo_campo': 'geom_risco_geologico_hidrologico' } })
dict_exemplos.append(
    {
        'identificador': 'geom_risco_geologico'
        , 'categorias': tuple(cat)
        , 'sheet_name': 'geom_risco_geologico'
        , 
    })

cat = []
cat.append({ 'Restrição Geotécnica': { 'camada' : 'restricao_geotecnica', 'prefixo_campo': 'geom_restricao_geotecnica' } })
dict_exemplos.append(
    {
        'identificador': 'geom_restricao_geotecnica'
        , 'categorias': tuple(cat)
        , 'sheet_name': 'geom_restricao_geotecnica'
        , 
    })

cat = []
cat.append({ 'Bem Tombado e/ou em Processo de Tombamento': { 'camada' : 'patrimonio_cultural_bem_tombado', 'prefixo_campo': 'geom_tombado' } })
dict_exemplos.append(
    {
        'identificador': 'geom_tombado'
        , 'categorias': tuple(cat)
        , 'sheet_name': 'geom_tombado'
        , 
    })

cat = []
cat.append({ 'Área Envoltória CONDEPHAAT': { 'camada' : 'patrimonio_cultural_area_envoltoria_CONDEPHAAT', 'prefixo_campo': 'geom_area_envoltoria_condephaat' } })
cat.append({ 'Área Envoltória CONPRESP': { 'camada' : 'patrimonio_cultural_area_envoltoria_CONPRESP', 'prefixo_campo': 'geom_area_envoltoria_condephaat' } })
cat.append({ 'Área Envoltória IPHAN': { 'camada' : 'patrimonio_cultural_area_envoltoria_IPHAN', 'prefixo_campo': 'geom_area_envoltoria_iphan' } })
dict_exemplos.append(
    {
        'identificador': 'geom_area_envoltoria'
        , 'categorias': tuple(cat)
        , 'sheet_name': 'geom_area_envoltoria'
        , 
    })

cat = []
cat.append({ 'Bairro Ambiental': { 'camada' : 'patrimonio_cultural_bairro_ambiental', 'prefixo_campo': 'geom_area_tombada_bairro_ambiental' } })
cat.append({ 'Lugar de Interesse Paisagístico Ambiental': { 'camada' : 'patrimonio_cultural_lugar_paisagistico_ambiental', 'prefixo_campo': 'geom_area_tombada_lugar_paisagistico_ambiental' } })
dict_exemplos.append(
    {
        'identificador': 'geom_area_tombada'
        , 'categorias': tuple(cat)
        , 'sheet_name': 'geom_area_tombada'
        , 
    })

cat = []
cat.append({ 'Preservações Culturais - Sítio Arqueológico': { 'camada' : 'patrimonio_cultural_sitio_arqueologico', 'prefixo_campo': 'geom_preservacoes_culturais_sitio_arqueologico' } })
cat.append({ 'Preservações Culturais - Ocorrência Arqueológica': { 'camada' : 'patrimonio_cultural_ocorrencia_arqueologica', 'prefixo_campo': 'geom_preservacoes_culturais_occ_arqueologica' } })
cat.append({ 'Preservações Culturais - Bem de Interesse Arqueológico': { 'camada' : 'patrimonio_cultural_bem_arqueologico', 'prefixo_campo': 'geom_preservacoes_culturais_beminter_arqueologico' } })
cat.append({ 'Preservações Culturais - Área de Interesse Arqueológico': { 'camada' : 'patrimonio_cultural_area_arqueologica', 'prefixo_campo': 'geom_preservacoes_culturais_arinter_arqueologico' } })
cat.append({ 'Preservações Culturais - Acervo Tombado': { 'camada' : 'patrimonio_cultural_acervo_tombado', 'prefixo_campo': 'geom_preservacoes_culturais_ac_tombado' } })
dict_exemplos.append(
    {
        'identificador': 'geom_outras_preservacoes_culturais'
        , 'categorias': tuple(cat)
        , 'sheet_name': 'geom_outras_preservacoes_culturais'
        , 
    })

cat = []
cat.append({ 'TAC Cultural': { 'camada' : None, 'prefixo_campo': 'geom_tac_cultural' } })
dict_exemplos.append(
    {
        'identificador': 'geom_tac_cultural'
        , 'categorias': tuple(cat)
        , 'sheet_name': 'geom_tac_cultural'
        , 
    })

cat = []
cat.append({ 'TAC Urbanistico': { 'camada' : None, 'prefixo_campo': 'geom_tac_urbanistico' } })
dict_exemplos.append(
    {
        'identificador': 'geom_tac_urbanistico'
        , 'categorias': tuple(cat)
        , 'sheet_name': 'geom_tac_urbanistico'
        , 
    })

cat = []
cat.append({ 'Registro Profissional': { 'camada' : None, 'prefixo_campo': 'registro_profissional' } })
dict_exemplos.append(
    {
        'identificador': 'registro_profissional'
        , 'categorias': tuple(cat)
        , 'sheet_name': 'registro_profissional'
        , 
    })

cat = []
cat.append({ 'Responsabilidade Técnica': { 'camada' : None, 'prefixo_campo': 'doc_responsabilidade_tecnica' } })
dict_exemplos.append(
    {
        'identificador': 'doc_responsabilidade_tecnica'
        , 'categorias': tuple(cat)
        , 'sheet_name': 'doc_responsabilidade_tecnica'
        , 
    })

cat = []
cat.append({ 'Autorização para bancas de jornal': { 'camada' : None, 'prefixo_campo': 'geom_intervencoes_passeio_bancas' } })
cat.append({ 'Autorização para uso do passeio': { 'camada' : None, 'prefixo_campo': 'geom_intervencoes_passeio_uso' } })
cat.append({ 'Obras no passeio': { 'camada' : None, 'prefixo_campo': 'geom_intervencoes_passeio_obras' } })
dict_exemplos.append(
    {
        'identificador': 'geom_intervencoes_passeio'
        , 'categorias': tuple(cat)
        , 'sheet_name': 'geom_intervencoes_passeio'
        , 
    })

cat = []
cat.append({ 'Embargos de obra': { 'camada' : None, 'prefixo_campo': 'geom_embargos' } })
cat.append({ 'Interdições de imóvel': { 'camada' : None, 'prefixo_campo': 'geom_interdicoes' } })
dict_exemplos.append(
    {
        'identificador': 'geom_embargos_interdicoes'
        , 'categorias': tuple(cat)
        , 'sheet_name': 'geom_embargos_interdicoes'
        , 
    })

cat = []
cat.append({ 'CAB': { 'camada' : None, 'prefixo_campo': 'CAB' } })
dict_exemplos.append(
    {
        'identificador': 'CAB'
        , 'categorias': tuple(cat)
        , 'sheet_name': 'CAB'
        , 
    })

In [19]:
import pandas as pd
import numpy as np
import time
import datetime
import openpyxl

In [20]:
messages = []

In [21]:
def escreve_excel(dfs, arquivo):
    with pd.ExcelWriter(arquivo) as writer:  
        for k, v in dfs.items():
            (v).to_excel(writer, sheet_name = k, index=False)
    # df.to_excel(arquivo, sheet_name=planilha, index = False)

In [22]:
def trata_coluna(col, id_col, index, camada, identificador, categoria, prefixo_campo, totalFeatures, numberMatched, numberReturned, dth_extracao):
    ds= []
    dfCol = pd.DataFrame(
        columns = ['Id', 'identificador', 'categoria', 'camada', 'prefixo_campo','totalFeatures','numberMatched','numberReturned','dth_extracao', 'atributo', 'flg_chave', 'flg_nullable', 'datatype', 'qtd_distinct_values']    
    )
    
    dfColDePara = pd.DataFrame(
        columns = ['Id', 'identificador', 'categoria', 'camada', 'prefixo_campo','totalFeatures','numberMatched','numberReturned','dth_extracao', 'atributo', 'flg_chave', 'flg_nullable', 'datatype', 'qtd_distinct_values', 'valor_de', 'valor_para']    
    )
    key = 'Sim' if col == id_col else 'Não'
    prop_dict = (schemas[camada])[col]
    nullable = 'Sim' if prop_dict['nullable'] else 'Não'
    datatype =  prop_dict['dtype']

    if key == 'Sim':
        arr = np.array([index, '-'])
        df = pd.DataFrame([arr], columns=['Id', 'valor_de'])            
        qtd_distinct_values = totalFeatures
    elif col == 'ge_multipoligono' or col == 'ge_poligono' or col == 'ge_linha' or col == 'ge_ponto':
        arr = np.array([index, '-'])
        df = pd.DataFrame([arr], columns=['Id', 'valor_de'])            
        qtd_distinct_values = totalFeatures
    else:
        distinct_arr = []
        # k = 0
        for v in features:
            """
            k += 1
            if k % 50000 == 0:
                print(f"{k} of {totalFeatures}: {(k / totalFeatures):%}")                
            
            if col == 'ge_multipoligono' or col == 'ge_poligono' or col == 'ge_linha' or col == 'ge_ponto':
                try:
                    distinct_arr.append([index, item] for item in ((v['geometry'])['coordinates']))
                except:
                    distinct_arr.append([index, None])
            else:
                distinct_arr.append([index, (v['properties'])[col]])
            """

            distinct_arr.append([index, (v['properties'])[col]])
        try:
            df = pd.DataFrame(distinct_arr, columns=['Id', 'valor_de']) 
            df.drop_duplicates(keep='first', inplace=True)  #dropa os duplicados sem copiar para outro objeto mantendo a primeira observação do mesmo
            qtd_distinct_values = df.shape[0]
        except Exception as error:
            messages.append(f"### Erro ao deduplicar dados {camada} - {col}: {type(error).__name__} – {error} ! ### </ br>")
            arr = np.array([index, '-'])
            df = pd.DataFrame([arr], columns=['Id', 'valor_de'])
            qtd_distinct_values = totalFeatures
        
    ds.append([index, identificador, categoria, camada, prefixo_campo, totalFeatures, numberMatched, numberReturned, dth_extracao, col, key, nullable, datatype, qtd_distinct_values])
    dfCol = pd.DataFrame(ds, columns=['Id', 'identificador', 'categoria', 'camada', 'prefixo_campo','totalFeatures','numberMatched','numberReturned','dth_extracao', 'atributo', 'flg_chave', 'flg_nullable', 'datatype', 'qtd_distinct_values'])
    dfColDePara = pd.DataFrame(ds, columns=['Id', 'identificador', 'categoria', 'camada', 'prefixo_campo','totalFeatures','numberMatched','numberReturned','dth_extracao', 'atributo', 'flg_chave', 'flg_nullable', 'datatype', 'qtd_distinct_values'])
    
    pct_distinct_values = qtd_distinct_values / totalFeatures
    var_discreta = key == 'Sim' or (pct_distinct_values > 0.94999 and totalFeatures > 500) or col == 'ge_multipoligono' or col == 'ge_poligono' or col == 'ge_linha'
    
    empty_values = var_discreta == True or qtd_distinct_values == 0 or qtd_distinct_values > 1000 
            
    if empty_values:
        dfColDePara = dfCol.assign(valor_de = "", valor_para = "")

        dfCol.drop(["Id"], axis=1, inplace=True)
        dfColDePara.drop(["Id"], axis=1, inplace=True)
    else:            
        # dfValues = pd.DataFrame(distinct_values, columns=["valor_de"])
        # l = pd.concat([dfValues.assign(Id=row.Id) for _, row in dfColDePara.iterrows()])

        #adiciona um sufixo qualquer para a coluna adicional para usar como busca de coluan a ser removida
        # remove as colunas do merge adicionais usadas como chave        
        dfTmp = pd.merge(dfCol, df, how="inner", on=["Id", "Id"], suffixes=('_xykey_x', '_xykey_y'))
        dfTmp.drop([i for i in dfTmp.columns if '_xykey_' in i], axis=1, inplace=True)        
        dfColDePara = dfTmp.assign(valor_para = "")
        
        dfCol.drop(["Id"], axis=1, inplace=True)
        dfColDePara.drop(["Id"], axis=1, inplace=True)        
        
    # print(f"shape {col} ===> dfCol: {dfCol.shape} | dfColDePara: {dfColDePara.shape}")
    return dfCol, dfColDePara        

In [23]:
dfs = {}
dfsDP = {}

for ex in dict_exemplos:    
    identificador = ex['identificador']
    planilha = ex['sheet_name']
    print(f"exemplo: {identificador} - planilha: {planilha} {datetime.datetime.now():%Y-%m-%d %H:%M:%S}")
    ds = []
    df = pd.DataFrame(
        columns = ['identificador', 'categoria', 'camada', 'prefixo_campo','totalFeatures','numberMatched','numberReturned','dth_extracao', 'atributo', 'flg_chave', 'flg_nullable', 'datatype', 'qtd_distinct_values']    
    )
    dfDP = pd.DataFrame(
        columns = ['identificador', 'categoria', 'camada', 'prefixo_campo','totalFeatures','numberMatched','numberReturned','dth_extracao', 'atributo', 'flg_chave', 'flg_nullable', 'datatype', 'qtd_distinct_values', 'valor_de', 'valor_para']    
    )
    for item in ex['categorias']: 
       for categoria in item.keys():
           camada = (item[categoria])['camada']
           prefixo_campo = (item[categoria])['prefixo_campo']
           
           features = totalFeatures = numberMatched = numberReturned = ts = dth_extracao = col = None
           key = nullable = 'Não'
           datatype = distinct_values = None
           qtd_distinct_values = 0

           if camada:
               retorno = geosampa.get_feature(camada)
               features = retorno['features']
               totalFeatures = retorno['totalFeatures']
               numberMatched = retorno['numberMatched']
               numberReturned = retorno['numberReturned']
               
               ts = retorno['timeStamp'] # exemplo: #2024-08-10T02:53:43.421Z
               #dth_extracao = time.mktime(datetime.datetime.strptime(ts,"%Y-%m-%dT%H:%M:%S.%fZ").timetuple()) # não salvar como unix timestamp, mas sim como datetime

               dth_extracao = datetime.datetime.strftime(datetime.datetime.strptime(ts,"%Y-%m-%dT%H:%M:%S.%fZ"), "%Y-%m-%d %H:%M")
               
               id_col = (schemas[camada])['id_col'] if 'id_col' in schemas[camada] else '' 
               idx = 1
               for col in schemas[camada].keys():
                   if col != 'id_col':
                       try:
                           # %time dfCol, dfColDePara = trata_coluna(col, id_col, idx, camada, identificador, categoria, prefixo_campo, totalFeatures, numberMatched, numberReturned, dth_extracao)

                           dfCol, dfColDePara = trata_coluna(col, id_col, idx, camada, identificador, categoria, prefixo_campo, totalFeatures, numberMatched, numberReturned, dth_extracao)
                           
                           df = pd.concat([df, dfCol])
                           dfDP = pd.concat([dfDP, dfColDePara])
                       except Exception as error:
                           messages.append(f"### Erro ao processar {camada} - {col}: {type(error).__name__} – {error} ! ###")
                   idx += 1
           break
    
    dfs[planilha] = df
    dfsDP[planilha] = dfDP

exemplo: geom_macrozoneamento - planilha: geom_macrozoneamento 2024-08-30 18:41:36
exemplo: geom_zeis-pde - planilha: geom_zeis-pde 2024-08-30 18:41:43
exemplo: geom_zoneamento - planilha: geom_zoneamento 2024-08-30 18:41:44
Paginação iniciada


C:\Users\leona\AppData\Local\Temp\ipykernel_42424\1456900941.py:22: UserWarning: Paginação iniciada. Serão realizadas 3 requisições para a zoneamento_2016_map1
  warnings.warn(f"Paginação iniciada. Serão realizadas {total_steps} requisições para a {feature_name}")


exemplo: geom_operacao_urbana - planilha: geom_operacao_urbana 2024-08-30 18:42:26
exemplo: geom_aiu - planilha: geom_aiu 2024-08-30 18:42:26
exemplo: geom_leis_esparsas - planilha: geom_leis_esparsas 2024-08-30 18:42:27
exemplo: geom_subprefeitura - planilha: geom_subprefeitura 2024-08-30 18:42:29
exemplo: geom_distrito - planilha: geom_distrito 2024-08-30 18:42:30
exemplo: geom_melhoramento_viario - planilha: geom_melhoramento_viario 2024-08-30 18:42:31
exemplo: geom_dis_dup - planilha: geom_dis_dup 2024-08-30 18:42:33
exemplo: geom_faixas_dominio - planilha: geom_faixas_dominio 2024-08-30 18:42:34
exemplo: geom_terra_indigena - planilha: geom_terra_indigena 2024-08-30 18:42:35
exemplo: geom_areas_publicas - planilha: geom_areas_publicas 2024-08-30 18:42:36
Paginação iniciada


C:\Users\leona\AppData\Local\Temp\ipykernel_42424\1456900941.py:22: UserWarning: Paginação iniciada. Serão realizadas 17 requisições para a calcada
  warnings.warn(f"Paginação iniciada. Serão realizadas {total_steps} requisições para a {feature_name}")


exemplo: geom_faixas_nao_edificaveis - planilha: geom_faixas_nao_edificaveis 2024-08-30 18:45:23
exemplo: geom_cartorio_registro_imoveis - planilha: geom_cartorio_registro_imoveis 2024-08-30 18:46:27
exemplo: geom_espaco_aereo - planilha: geom_espaco_aereo 2024-08-30 18:46:27
exemplo: geo_minianel_viario - planilha: geo_minianel_viario 2024-08-30 18:46:30
exemplo: geom_logradouro - planilha: geom_logradouro 2024-08-30 18:46:30
Paginação iniciada


...

Paginação iniciada


C:\Users\leona\AppData\Local\Temp\ipykernel_42424\1456900941.py:22: UserWarning: Paginação iniciada. Serão realizadas 10 requisições para a cobertura_vegetal
  warnings.warn(f"Paginação iniciada. Serão realizadas {total_steps} requisições para a {feature_name}")


Paginação iniciada


C:\Users\leona\AppData\Local\Temp\ipykernel_42424\1456900941.py:22: UserWarning: Paginação iniciada. Serão realizadas 22 requisições para a arvore
  warnings.warn(f"Paginação iniciada. Serão realizadas {total_steps} requisições para a {feature_name}")


exemplo: geom_biomas_protegidos - planilha: geom_biomas_protegidos 2024-08-30 19:10:15
exemplo: geom_tca - planilha: geom_tca 2024-08-30 19:10:19
exemplo: geom_tac_ambiental - planilha: geom_tac_ambiental 2024-08-30 19:10:21
exemplo: geom_supressao_irregular_vegetacao_arborea - planilha: geom_supressao_irregular_vegetacao_arborea 2024-08-30 19:10:21
exemplo: geom_area_contaminada - planilha: geom_area_contaminada 2024-08-30 19:10:21
exemplo: geom_servidao_ambiental - planilha: geom_servidao_ambiental 2024-08-30 19:10:22
exemplo: geom_servicos_ambientais - planilha: geom_servicos_ambientais 2024-08-30 19:10:22
exemplo: geom_espaço_aereo_ruido - planilha: geom_espaço_aereo_ruido 2024-08-30 19:10:22
exemplo: geom_risco_geologico - planilha: geom_risco_geologico 2024-08-30 19:10:22
exemplo: geom_restricao_geotecnica - planilha: geom_restricao_geotecnica 2024-08-30 19:10:22
exemplo: geom_tombado - planilha: geom_tombado 2024-08-30 19:10:22
exemplo: geom_area_envoltoria - planilha: geom_area

In [24]:
len(dfs)
len(dfsDP)

53

In [25]:
dh_arquivo = f"{datetime.datetime.now():%Y%m%d_%H%M}"

In [26]:
epath = f"{ROOT_DIR}/output/{dh_arquivo}_metadados_atributos.xlsx"
print(epath)
escreve_excel(dfs, epath)

F:\General Projects\Git\GitHub\sepep-pmsp\saade_governancadados/output/20240830_1910_metadados_atributos.xlsx


C:\ProgramData\Anaconda3\envs\geopandas\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [27]:
# escreve_excel(dfsDP, f"{dh_arquivo}_metadados_v4.xlsx")
epath = f"{ROOT_DIR}/output/{dh_arquivo}_metadados_v4.xlsx"
print(epath)
escreve_excel(dfsDP, epath)

F:\General Projects\Git\GitHub\sepep-pmsp\saade_governancadados/output/20240830_1910_metadados_v4.xlsx


In [28]:
print(messages)

["### Erro ao processar GEOSAMPA_heliponto - ge_ponto: ValueError – You are trying to merge on int64 and object columns for key 'Id'. If you wish to proceed you should use pd.concat ! ###", "### Erro ao processar patrimonio_cultural_sitio_arqueologico - ge_ponto: ValueError – You are trying to merge on int64 and object columns for key 'Id'. If you wish to proceed you should use pd.concat ! ###", "### Erro ao processar patrimonio_cultural_ocorrencia_arqueologica - ge_ponto: ValueError – You are trying to merge on int64 and object columns for key 'Id'. If you wish to proceed you should use pd.concat ! ###", "### Erro ao processar patrimonio_cultural_bem_arqueologico - ge_ponto: ValueError – You are trying to merge on int64 and object columns for key 'Id'. If you wish to proceed you should use pd.concat ! ###", "### Erro ao processar patrimonio_cultural_acervo_tombado - ge_ponto: ValueError – You are trying to merge on int64 and object columns for key 'Id'. If you wish to proceed you shou

In [29]:
print(f"Processamento concluído: {datetime.datetime.now():%d/%M</%Y %H:%m}")

Processamento concluído: 30/10</2024 19:08
